In [1]:
import ee
import geemap

## NAIP: National Agriculture Imagery Program

The National Agriculture Imagery Program (NAIP) acquires aerial imagery during the agricultural growing seasons in the continental U.S.

NAIP projects are contracted each year based upon available funding and the FSA imagery acquisition cycle. Beginning in 2003, NAIP was acquired on a 5-year cycle. 2008 was a transition year, and a three-year cycle began in 2009.

NAIP imagery is acquired at a **one-meter** ground sample distance (GSD) with a horizontal accuracy that matches within six meters of photo-identifiable ground control points, which are used during image inspection.

Older images were collected using 3 bands (Red, Green, and Blue: RGB), but newer imagery is usually collected with an additional near-infrared band (RGBN). 

More information about NAIP imagery can be found on [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ).

## Create annual composite of NAIP imagery 

Select 4-band (RGBN) NAIP imagery.

In [2]:
naip_ts = geemap.naip_timeseries(start_year=2009, end_year=2018)

Create a list of layer names to be shown under the dropdown list. 

In [3]:
layer_names = ['NAIP ' + str(year) for year in range(2009, 2019)]
print(layer_names)

['NAIP 2009', 'NAIP 2010', 'NAIP 2011', 'NAIP 2012', 'NAIP 2013', 'NAIP 2014', 'NAIP 2015', 'NAIP 2016', 'NAIP 2017', 'NAIP 2018']


Set visualization parameters.

In [4]:
naip_vis = {'bands': ['N', 'R', 'G']}

Create a split-panel map for visualizing NAIP imagery. Use the dropdown menus in the top corners to select the layer years.

In [5]:
Map = geemap.Map()
Map.ts_inspector(left_ts=naip_ts, right_ts=naip_ts, left_names=layer_names, right_names=layer_names, left_vis=naip_vis, right_vis=naip_vis)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=Dropdown(layout=Layout(max_width='…

## Create annual composite of Landsat imagery

Use the drawing tools to create an ROI

In [7]:
Input_Map = geemap.Map()
Input_Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [8]:
region = Input_Map.draw_last_feature
roi = region.geometry()

In [9]:
landsat_ts = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2019, start_date='01-01', end_date='12-31')

In [10]:
layer_names = ['Landsat ' + str(year) for year in range(1984, 2020)]
print(layer_names)

['Landsat 1984', 'Landsat 1985', 'Landsat 1986', 'Landsat 1987', 'Landsat 1988', 'Landsat 1989', 'Landsat 1990', 'Landsat 1991', 'Landsat 1992', 'Landsat 1993', 'Landsat 1994', 'Landsat 1995', 'Landsat 1996', 'Landsat 1997', 'Landsat 1998', 'Landsat 1999', 'Landsat 2000', 'Landsat 2001', 'Landsat 2002', 'Landsat 2003', 'Landsat 2004', 'Landsat 2005', 'Landsat 2006', 'Landsat 2007', 'Landsat 2008', 'Landsat 2009', 'Landsat 2010', 'Landsat 2011', 'Landsat 2012', 'Landsat 2013', 'Landsat 2014', 'Landsat 2015', 'Landsat 2016', 'Landsat 2017', 'Landsat 2018', 'Landsat 2019']


In [11]:
landsat_vis = {
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']}

In [13]:
Landsat_Map = geemap.Map()
Landsat_Map.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, left_vis=landsat_vis, right_vis=landsat_vis)
Landsat_Map.centerObject(roi, zoom=10)
Landsat_Map

Map(center=[37.69660400504876, -121.88238799999962], controls=(WidgetControl(options=['position'], widget=Drop…

## Compare Landsat imagery and National Land Cover Database (NLCD)

More information about NLCD can be found at the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD).

Use the drawing tools in the map below to select a region of interest.

In [22]:
Input_Map_2 = geemap.Map()
Input_Map_2

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

'float' object is not iterable


In [33]:
region = Input_Map_2.draw_last_feature
roi = region.geometry()

In [34]:
NLCD = ee.ImageCollection('USGS/NLCD')

In [35]:
NLCD_layers = NLCD.aggregate_array('system:id').getInfo()
print(NLCD_layers)

['USGS/NLCD/NLCD1992', 'USGS/NLCD/NLCD2001', 'USGS/NLCD/NLCD2001_AK', 'USGS/NLCD/NLCD2001_HI', 'USGS/NLCD/NLCD2001_PR', 'USGS/NLCD/NLCD2004', 'USGS/NLCD/NLCD2006', 'USGS/NLCD/NLCD2008', 'USGS/NLCD/NLCD2011', 'USGS/NLCD/NLCD2011_AK', 'USGS/NLCD/NLCD2011_HI', 'USGS/NLCD/NLCD2011_PR', 'USGS/NLCD/NLCD2013', 'USGS/NLCD/NLCD2016']


In [36]:
NLCD2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover').clip(roi)
NLCD2006 = ee.Image('USGS/NLCD/NLCD2006').select('landcover').clip(roi)
NLCD2011 = ee.Image('USGS/NLCD/NLCD2011').select('landcover').clip(roi)
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(roi)

In [37]:
NLCD = ee.ImageCollection([NLCD2001, NLCD2006, NLCD2011, NLCD2016])

In [38]:
NLCD_layer_names = ['NLCD ' + str(year) for year in range(2001, 2017, 5)]
print(NLCD_layer_names)

['NLCD 2001', 'NLCD 2006', 'NLCD 2011', 'NLCD 2016']


In [39]:
landsat_ts = geemap.landsat_timeseries(roi=roi, start_year=2001, end_year=2016, start_date='01-01', end_date='12-31')

In [40]:
landsat_layer_names = ['Landsat ' + str(year) for year in range(2001, 2017)]

In [41]:
landsat_vis = {
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']}

nlcd_vis = {
    'bands': ['landcover']
}

In [42]:
Landsat_NLCD_Map = geemap.Map()
Landsat_NLCD_Map.ts_inspector(left_ts=landsat_ts, right_ts=NLCD, left_names=landsat_layer_names, right_names=NLCD_layer_names, left_vis=landsat_vis, right_vis=nlcd_vis)
Landsat_NLCD_Map.centerObject(roi, zoom=8)
Landsat_NLCD_Map

Map(center=[39.943480175492475, -121.50171036596741], controls=(WidgetControl(options=['position'], widget=Dro…